In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import io
from tensorflow.python.keras import Model
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate, Dropout, Lambda, Reshape, AveragePooling2D, Flatten, Dense, Add, UpSampling2D
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import glob
import os
import pickle
import segmentation_models as sm
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, CSVLogger, TensorBoard
from tensorflow.python import keras

from tensorflow.python.keras import layers

import tensorflow_probability as tfp

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
image_feature_description = {
  'image': tf.io.FixedLenFeature([], tf.string),
  'label': tf.io.FixedLenFeature([], tf.string),
  }

In [ ]:
def _parse_image_function(example_proto):
    features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.io.decode_raw(features['image'], tf.float32)
    image.set_shape([16 * 224 * 224])
    image = tf.reshape(image, [224, 224,16])

    tensor1 = tf.convert_to_tensor((np.ones((224,224,16))*np.array([[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,
                                                                   0.02,0.02,0.02,0.02, 0.03, 0.01]])).astype(np.float32))
    tensor2 = tf.convert_to_tensor((np.ones((224,224,16))*np.array([[0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0]])).astype(np.float32))

    image = tf.multiply(image, tensor1)
    image = tf.add(image, tensor2)

    label = tf.io.decode_raw(features['label'], tf.float32)
    label.set_shape([1 * 224 * 224])
    label = tf.reshape(label, [ 224, 224,1])

    return image[:,:,:15], label   # image[:,:,10:15] = S1 + NL, image[:,:,:15] = S2 + S1 + NL,

In [ ]:
def read_dataset(epochs, batch_size, filenames):

    dataset = tf.data.TFRecordDataset(filenames)

    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string),
    }

    dataset = dataset.map(_parse_image_function)
    dataset = dataset.prefetch(64)
    #dataset = dataset.repeat(epochs)
    #dataset = dataset.shuffle(buffer_size=2 * batch_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [ ]:
ListBackbone = ['inceptionv3','vgg19','resnet18','mobilenetv2']

In [ ]:
ListBackbone = ['inceptionv3','vgg19','resnet18']

In [ ]:
ListBackbone = ['mobilenetv2']

In [ ]:
Berlin = '/dir/MultiBands2//Test00.tfrecords'
London = '/dir/MultiBands2//Test01.tfrecords'
testset = London
Len_test = sum(1 for _ in tf.data.TFRecordDataset(testset))
test_dataset = read_dataset(1, 1, testset)

In [ ]:
textfile = open('/dir/MultiBands2/Results/Unet_S1_NL_Berlin_MAE.csv', 'w')
textfile.write( 'Backbone/Fold')

for i in range(5) :
    textfile.write( ' , ')
    textfile.write( 'Fold'+str(i) )
textfile.write('\n')
textfile.close()

textfile2 = open('/dir/MultiBands2/Results/Unet_S1_NL_Berlin_MSE.csv', 'w')
textfile2.write( 'Backbone/Test')

for i in range(5) :
    textfile2.write( ' , ')
    textfile2.write( 'Fold'+str(i) )
textfile2.write('\n')
textfile2.close()

for backbone in ListBackbone:
    textfile = open('/dir/MultiBands2/Results/Unet_S1_NL_Berlin_MAE.csv', 'a')
    textfile2 =  open('/dir/MultiBands2/Results/Unet_S1_NL_Berlin_MSE.csv', 'a')
    textfile.write( backbone + ' , ')
    textfile2.write( backbone + ' , ')


    for i in range(5):
        model = tf.keras.models.load_model('/dir/MultiBands2/Models/Unet_' + backbone + '_S1_NL_' + str(i))
        model.compile( loss='mean_squared_error', optimizer='adam')

        Y_predict = np.zeros((Len_test,224,224))
        Y_train =  np.zeros((Len_test,224,224))
        count = 0
        for p, l in test_dataset.take(Len_test): # Takes 1 batch
            y_hat = model(p)
            y_r = l

            Y_predict[count,:,:] = y_hat[:,:,:,0].numpy()
            Y_train[count,:,:] = y_r[:,:,:,0].numpy()

            count = count + 1

        test_MSE = mean_squared_error(Y_train.reshape(Len_test*224*224), Y_predict.reshape(Len_test*224*224))
        test_R = mean_absolute_error(Y_train.reshape(Len_test*224*224), Y_predict.reshape(Len_test*224*224))
        print('test_MSE= '+ str(test_MSE))
        print('test_R= '+ str(test_R))

        textfile.write( str(test_R) + ' , ')
        textfile2.write( str(test_MSE) + ' , ')

    textfile.write('\n')
    textfile2.write('\n')
    textfile.close()
    textfile2.close()

test_MSE= 14.856551260858218
test_R= 1.5996266746355856
test_MSE= 16.956415610668714
test_R= 1.683934767158241
test_MSE= 15.098048076734576
test_R= 1.7393267422550045
test_MSE= 15.13943646558708
test_R= 1.6735123915123795
test_MSE= 15.501804746936264
test_R= 1.734999574697693


In [ ]:
textfile = open('/dir/MultiBands2/Results/Unet_S1_S2_NL_Berlin_MAE.csv', 'w')
textfile.write( 'Backbone/Fold')

for i in range(5) :
    textfile.write( ' , ')
    textfile.write( 'Fold'+str(i) )
textfile.write('\n')
textfile.close()

textfile2 = open('/dir/MultiBands2/Results/Unet_S1_S2_NL_Berlin_MSE.csv', 'w')
textfile2.write( 'Backbone/Test')

for i in range(5) :
    textfile2.write( ' , ')
    textfile2.write( 'Fold'+str(i) )
textfile2.write('\n')
textfile2.close()

for backbone in ListBackbone:
    textfile = open('/dir/MultiBands2/Results/Unet_S1_S2_NL_Berlin_MAE.csv', 'a')
    textfile2 =  open('/dir/MultiBands2/Results/Unet_S1_S2_NL_Berlin_MSE.csv', 'a')
    textfile.write( backbone + ' , ')
    textfile2.write( backbone + ' , ')


    for i in range(5):
        model = tf.keras.models.load_model('/dir/MultiBands2/Models/Unet_' + backbone + '_S1_S2_NL_' + str(i))
        model.compile( loss='mean_squared_error', optimizer='adam')

        Y_predict = np.zeros((Len_test,224,224))
        Y_train =  np.zeros((Len_test,224,224))
        count = 0
        for p, l in test_dataset.take(Len_test): # Takes 1 batch
            y_hat = model(p)
            y_r = l

            Y_predict[count,:,:] = y_hat[:,:,:,0].numpy()
            Y_train[count,:,:] = y_r[:,:,:,0].numpy()

            count = count + 1

        test_MSE = mean_squared_error(Y_train.reshape(Len_test*224*224), Y_predict.reshape(Len_test*224*224))
        test_R = mean_absolute_error(Y_train.reshape(Len_test*224*224), Y_predict.reshape(Len_test*224*224))
        print('test_MSE= '+ str(test_MSE))
        print('test_R= '+ str(test_R))

        textfile.write( str(test_R) + ' , ')
        textfile2.write( str(test_MSE) + ' , ')

    textfile.write('\n')
    textfile2.write('\n')
    textfile.close()
    textfile2.close()

test_MSE= 12.875762260275227
test_R= 1.4809710204837903
test_MSE= 13.031059113776019
test_R= 1.4603006160191432
test_MSE= 12.492405948801265
test_R= 1.4873980898506869
test_MSE= 14.330973345812653
test_R= 1.5222997800242815
test_MSE= 13.113847689230274
test_R= 1.5402506934175377


In [ ]:
textfile = open('/dir/MultiBands2/Results/Unet_S1_NL_London_MAE.csv', 'w')
textfile.write( 'Backbone/Fold')

for i in range(5) :
    textfile.write( ' , ')
    textfile.write( 'Fold'+str(i) )
textfile.write('\n')
textfile.close()

textfile2 = open('/dir/MultiBands2/Results/Unet_S1_NL_London_MSE.csv', 'w')
textfile2.write( 'Backbone/Test')

for i in range(5) :
    textfile2.write( ' , ')
    textfile2.write( 'Fold'+str(i) )
textfile2.write('\n')
textfile2.close()

for backbone in ListBackbone:
    textfile = open('/dir/MultiBands2/Results/Unet_S1_NL_London_MAE.csv', 'a')
    textfile2 =  open('/dir/MultiBands2/Results/Unet_S1_NL_London_MSE.csv', 'a')
    textfile.write( backbone + ' , ')
    textfile2.write( backbone + ' , ')


    for i in range(5):
        model = tf.keras.models.load_model('/dir/MultiBands2/Models/Unet_' + backbone + '_S1_NL_' + str(i))
        model.compile( loss='mean_squared_error', optimizer='adam')

        Y_predict = np.zeros((Len_test,224,224))
        Y_train =  np.zeros((Len_test,224,224))
        count = 0
        for p, l in test_dataset.take(Len_test): # Takes 1 batch
            y_hat = model(p)
            y_r = l

            Y_predict[count,:,:] = y_hat[:,:,:,0].numpy()
            Y_train[count,:,:] = y_r[:,:,:,0].numpy()

            count = count + 1

        test_MSE = mean_squared_error(Y_train.reshape(Len_test*224*224), Y_predict.reshape(Len_test*224*224))
        test_R = mean_absolute_error(Y_train.reshape(Len_test*224*224), Y_predict.reshape(Len_test*224*224))
        print('test_MSE= '+ str(test_MSE))
        print('test_R= '+ str(test_R))

        textfile.write( str(test_R) + ' , ')
        textfile2.write( str(test_MSE) + ' , ')

    textfile.write('\n')
    textfile2.write('\n')
    textfile.close()
    textfile2.close()

test_MSE= 11.461108121002528
test_R= 1.6911863505698816
test_MSE= 11.848014748281626
test_R= 1.7081063047292764
test_MSE= 11.745130366831722
test_R= 1.8831637964039138
test_MSE= 11.583060725603861
test_R= 1.7664039327487573
test_MSE= 11.60820141748073
test_R= 1.7948574344487374


In [ ]:

textfile = open('/dir/MultiBands2/Results/Unet_S1_S2_NL_London_MAE.csv', 'w')
textfile.write( 'Backbone/Fold')

for i in range(5) :
    textfile.write( ' , ')
    textfile.write( 'Fold'+str(i) )
textfile.write('\n')
textfile.close()

textfile2 = open('/dir/MultiBands2/Results/Unet_S1_S2_NL_London_MSE.csv', 'w')
textfile2.write( 'Backbone/Test')

for i in range(5) :
    textfile2.write( ' , ')
    textfile2.write( 'Fold'+str(i) )
textfile2.write('\n')
textfile2.close()

for backbone in ListBackbone:
    textfile = open('/dir/MultiBands2/Results/Unet_S1_S2_NL_London_MAE.csv', 'a')
    textfile2 =  open('/dir/MultiBands2/Results/Unet_S1_S2_NL_London_MSE.csv', 'a')
    textfile.write( backbone + ' , ')
    textfile2.write( backbone + ' , ')


    for i in range(5):
        model = tf.keras.models.load_model('/dir/MultiBands2/Models/Unet_' + backbone + '_S1_S2_NL_' + str(i))
        model.compile( loss='mean_squared_error', optimizer='adam')

        Y_predict = np.zeros((Len_test,224,224))
        Y_train =  np.zeros((Len_test,224,224))
        count = 0
        for p, l in test_dataset.take(Len_test): # Takes 1 batch
            y_hat = model(p)
            y_r = l

            Y_predict[count,:,:] = y_hat[:,:,:,0].numpy()
            Y_train[count,:,:] = y_r[:,:,:,0].numpy()

            count = count + 1

        test_MSE = mean_squared_error(Y_train.reshape(Len_test*224*224), Y_predict.reshape(Len_test*224*224))
        test_R = mean_absolute_error(Y_train.reshape(Len_test*224*224), Y_predict.reshape(Len_test*224*224))
        print('test_MSE= '+ str(test_MSE))
        print('test_R= '+ str(test_R))

        textfile.write( str(test_R) + ' , ')
        textfile2.write( str(test_MSE) + ' , ')

    textfile.write('\n')
    textfile2.write('\n')
    textfile.close()
    textfile2.close()

test_MSE= 10.346511586504464
test_R= 1.5695600322010852
test_MSE= 10.555392778170848
test_R= 1.5823340145600455
test_MSE= 10.38797545881581
test_R= 1.623212949852897
test_MSE= 10.945781224253379
test_R= 1.5688647302819079
test_MSE= 10.540995077274275
test_R= 1.6152345194930706
